# Análisis de datos utilizando PyRoot


In [6]:
import ROOT
import sys
import os

In [12]:
cwd = os.getcwd()
print cwd

/home/camilo/Proyectos/MadGraph/Notebooks


In [15]:
os.chdir("/home/camilo/Proyectos/MadGraph/Ejemplos_Clase/InclusiveTTbar1J")

In [17]:
ROOT.gSystem.Load("Delphes/libDelphes.so")
MainChain=ROOT.TChain("Delphes")

In [21]:
os.listdir("/home/camilo/Proyectos/MadGraph/Ejemplos_Clase/InclusiveTTbar1J")

['HTML',
 'Cards',
 'SubProcesses',
 'Source',
 'TemplateVersion.txt',
 'MGMEVersion.txt',
 'madevent.tar.gz',
 'bin',
 'Events',
 'README.systematics',
 'README',
 'index.html',
 'lib',
 'crossx.html']

In [ ]:
MainChain.Add("delphes_events.root")
MainChain.Draw("Jet_size >> NumeroDeJets(10,0,10)","")